In [17]:
import os
import logging
import pandas as pd
import json

# Import the necessary classes and functions from the new structure
from fits_metadata_extractor.processor import FITSProcessor
from fits_metadata_extractor.utils import save_metadata_to_csv, load_metadata_from_csv
from fits_metadata_extractor.search import search_fits_by_point, search_fits_by_region
from fits_metadata_extractor.plotter import plot_moc_and_polygon_from_dataset_notebook

# Configure logging to display INFO level messages
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [18]:
# Specify the directory containing FITS files
fits_directory = "fits_collection"  # <-- CHANGE THIS PATH to your FITS files directory
output_csv = 'metadata.csv'
plot_output_dir = 'plots'  # Directory where plots will be saved

# Check if the FITS directory exists
if not os.path.isdir(fits_directory):
    logging.error(f"The specified FITS directory does not exist: {fits_directory}")
else:
    # Process FITS files using parallel processing
    logging.info("Starting processing of FITS files.")
    metadata_df = process_fits_directory_parallel(fits_directory, max_workers=5)
    logging.info("Finished processing FITS files.")


2024-10-23 15:16:52,403 - INFO - Starting processing of FITS files.
2024-10-23 15:16:52,418 - INFO - Processing file: fits_collection/G327.617-0.364_I4.fits
2024-10-23 15:16:52,419 - INFO - Processing file: fits_collection/SN1987A_cut_35_smHB.fits
2024-10-23 15:16:52,419 - INFO - Processing file: fits_collection/G351.702+0.672_atlasgal.fits
2024-10-23 15:16:52,419 - INFO - Processing file: fits_collection/5GHz_n_f.fits
2024-10-23 15:16:52,420 - INFO - Processing file: fits_collection/G351.632-0.459_atlasgal.fits
2024-10-23 15:16:52,505 - INFO - Inferred object name 'G327.617-0.364' for file 'fits_collection/G327.617-0.364_I4.fits'.
2024-10-23 15:16:52,513 - INFO - Successfully processed 'fits_collection/SN1987A_cut_35_smHB.fits'. Resolved using VizieR: 'SNR1987A'. MOC Details - Sky Fraction: 0.000000, Number of Cells: 3, Max Order: 10. Polygon: POLYGON ((83.98078838261291 -69.31003927205, 83.75255161738708 -69.31003927205, 83.75297519652439 -69.22940072795001, 83.98036480347561 -69.229

In [19]:
# Save the metadata to CSV
save_metadata_to_csv(metadata_df, output_csv)
logging.info(f"Metadata saved to {output_csv}.")


2024-10-23 15:16:56,108 - INFO - Metadata successfully saved to metadata.csv.
2024-10-23 15:16:56,109 - INFO - Metadata saved to metadata.csv.


In [20]:
# Plotting all FITS files with a limit for testing purposes
try:
    logging.info("Starting plotting of MOCs and polygons.")
    plot_moc_and_polygon_from_dataset_notebook(
        metadata_df=metadata_df,
        input_dir=fits_directory,
        output_dir=plot_output_dir,
        max_plots=2  # Adjust as needed; set to None to plot all
    )
    logging.info("Finished plotting of MOCs and polygons.")
except Exception as e:
    logging.error(f"Error occurred during plotting: {e}")


2024-10-23 15:16:58,761 - INFO - Starting plotting of MOCs and polygons.
2024-10-23 15:16:58,762 - INFO - Plots will be saved to: plots
2024-10-23 15:16:58,762 - INFO - No specific selection criteria provided. All FITS files will be plotted.
2024-10-23 15:16:58,763 - INFO - Generating 2 plots.


Generating Plots:   0%|          | 0/2 [00:00<?, ?it/s]

2024-10-23 15:16:58,796 - INFO - Calling _reproject_full in non-dask mode
2024-10-23 15:17:00,115 - INFO - Plot saved as SN1987A_cut_35_smHB.png
2024-10-23 15:17:00,116 - INFO - Plot saved: plots/SN1987A_cut_35_smHB.png
2024-10-23 15:17:00,138 - INFO - Calling _reproject_full in non-dask mode
2024-10-23 15:17:01,323 - INFO - Plot saved as G327.617-0.364_I4.png
2024-10-23 15:17:01,323 - INFO - Plot saved: plots/G327.617-0.364_I4.png
2024-10-23 15:17:01,324 - INFO - Finished plotting of MOCs and polygons.


In [21]:
# Define test inputs for search by point
test_point = {
    'ra': 150.0,  # Right Ascension in degrees
    'dec': 2.2    # Declination in degrees
}

logging.info(f"Testing search by point: RA={test_point['ra']}, Dec={test_point['dec']}")
matching_fits_by_point = search_fits_by_point(metadata_df, test_point['ra'], test_point['dec'])

if not matching_fits_by_point.empty:
    logging.info(f"FITS files containing the point (RA={test_point['ra']}, Dec={test_point['dec']}):")
    display(matching_fits_by_point[['FITS_File', 'Resolved_Object']])
else:
    logging.info("No FITS files found containing the specified point.")


2024-10-23 15:17:11,820 - INFO - Testing search by point: RA=150.0, Dec=2.2
2024-10-23 15:17:11,825 - INFO - FITS files containing the point (RA=150.0, Dec=2.2):


,FITS_File,Resolved_Object
11,id12_GAL-Survey-GC.fpsf.fits,Unknown


In [22]:
# Define test inputs for search by circular region
test_circle = {
    'type': 'circle',
    'center': (150.0, 2.2),  # Center RA and Dec in degrees
    'radius': 1.0             # Radius in degrees
}

logging.info(f"Testing search by circular region: Center RA={test_circle['center'][0]}, Dec={test_circle['center'][1]}, Radius={test_circle['radius']} degrees")
matching_fits_by_circle = search_fits_by_region(metadata_df, test_circle)

if not matching_fits_by_circle.empty:
    logging.info(f"FITS files intersecting the circular region (RA={test_circle['center'][0]}, Dec={test_circle['center'][1]}, Radius={test_circle['radius']} degrees):")
    display(matching_fits_by_circle[['FITS_File', 'Resolved_Object']])
else:
    logging.info("No FITS files found intersecting the specified circular region.")


2024-10-23 15:17:22,870 - INFO - Testing search by circular region: Center RA=150.0, Dec=2.2, Radius=1.0 degrees
2024-10-23 15:17:22,876 - INFO - FITS files intersecting the circular region (RA=150.0, Dec=2.2, Radius=1.0 degrees):


,FITS_File,Resolved_Object
8,2013.1.01292.S_SB_X4eb_GB_X4ec_MB_X4ed_2-38011...,Unknown
11,id12_GAL-Survey-GC.fpsf.fits,Unknown
15,2013.1.00034.S_SB_X5_GB_X6_MB_X7_midz_cell10_2...,Unknown


In [23]:
# Define test inputs for search by polygonal region
test_polygon = {
    'type': 'polygon',
    'coordinates': [
        (149.0, 1.0),
        (151.0, 1.0),
        (151.0, 3.0),
        (149.0, 3.0)
    ]
}

logging.info(f"Testing search by polygonal region with coordinates: {test_polygon['coordinates']}")
matching_fits_by_polygon = search_fits_by_region(metadata_df, test_polygon)

if not matching_fits_by_polygon.empty:
    logging.info("FITS files intersecting the polygonal region:")
    display(matching_fits_by_polygon[['FITS_File', 'Resolved_Object']])
else:
    logging.info("No FITS files found intersecting the specified polygonal region.")


2024-10-23 15:17:25,106 - INFO - Testing search by polygonal region with coordinates: [(149.0, 1.0), (151.0, 1.0), (151.0, 3.0), (149.0, 3.0)]
2024-10-23 15:17:25,110 - INFO - FITS files intersecting the polygonal region:


,FITS_File,Resolved_Object
8,2013.1.01292.S_SB_X4eb_GB_X4ec_MB_X4ed_2-38011...,Unknown
11,id12_GAL-Survey-GC.fpsf.fits,Unknown
15,2013.1.00034.S_SB_X5_GB_X6_MB_X7_midz_cell10_2...,Unknown
